In [ ]:
import sys
sys.path.append('../python_packages_static/')
import pyemu
import os
import glob
import shutil

# We want to set up for global sensitivity analysis. 

In [ ]:
pst_root = 'prior_mc_wide'

In [ ]:
run_ensemble=False

In [ ]:
if run_ensemble==True:
    pst = pyemu.Pst(f'../noptmax0_testing/{pst_root}.pst')
else:
    pst = pyemu.Pst(f'../output/noptmax0/{pst_root}.pst')

output_dir = '../run_data'

### set `tie_by_group` to `True`. Also update the lower bound for CHD parameters to nearly 1.0 (because many of these values are at or near the bottom of model cells and if sampling sets them below the bottom of the cell, MODFLOW6 will not run). Also unfix the CHD parameters so they will be evaluated. All other defailts. for `pestpp-sen` will be accepted.

In [ ]:
pst.pestpp_options['tie_by_group'] = True
pst.parameter_data.loc[pst.parameter_data.pargp=='chd', 'parlbnd'] = 0.999999
pst.parameter_data.partrans = 'log'
pst.parameter_data.partrans.unique()

In [ ]:
pst.write(f'../run_data/{pst_root}_sens.pst')

## If `run_ensemble=True` the cell below will run a local `prior_mc_wide_sens` global sensitivity analysis
* **NOTE: must have the `pestpp-sen` executable in system path or in `../run_data/`**
* same process as in notebooks 1.0 and 1.3 for parallel run
* for this `pestpp-sen`run, the total number of model runs is 64, which is = # of parameter groups * 4
* will run in parallel locally using the number of cores specified below by `num_workers`
* creates a new directory called `"../master_sen/"` which is a copy of run_data
* while running generates worker directories that are removed when run is complete
* results moved to `"../run_data/"`

In [ ]:
if run_ensemble==True:
    #  set some variables for starting a group of PEST++ workers on the local machine
    # MAKE SURE THAT PESTPP-IES and MF6 executables are in your system path or are in '../run_data'
    
    num_workers = 5  # number of local workers -- VERY IMPORTANT, DO NOT MAKE TOO BIG
    if sys.platform == 'win32':
        pst_exe = 'pestpp-sen.exe'
    else:
        pst_exe = 'pestpp-sen'
        
    template_ws =  '../run_data'  #  template_directory
    m_d = '../master_sen'
    
    pyemu.os_utils.start_workers(worker_dir=template_ws, 
                             exe_rel_path=pst_exe,
                             pst_rel_path=f'{pst_root}_sens.pst',
                             num_workers=num_workers, 
                             master_dir=m_d
                            )

In [ ]:
if run_ensemble==True:
    #  move results into run_data and clean up 
    
    move_result_files = glob.glob(os.path.join(m_d, f'{pst_root}_sens*'))
    move_result_files = [f for f in move_result_files if 'pst' not in f]
    
    [shutil.copy(os.path.join(m_d, file), output_dir) for file in move_result_files]
    
    #  Remove master dir.
    shutil.rmtree(m_d)